## Assignment 3
임의의 이미지 데이터를 이용하여 ViT 모델의 전처리 과정이 CNN 모델과 어떻게 다른지 코드를 통해 비교하여 구현하세요.

- Requirements
1. C++ 또는 Python 이용하여 작성하세요.
2. TensorFlow, Pytorch, JAX 등 딥러닝 프레임워크 사용 가능 (Keras와 같은 상대적으로 고수준의 API는 사용 불가)
3. 이미지 로딩 및 임베딩을 얻기 위한 라이브러리 외에 사용 불가능

설명 ) 이미지 처리하는 방식이 다름.

- CNN모델
  - 이미지를 전체적으로 처리.
  - 이미지의 각 픽셀을 입력으로 받아 여러 계층의 convolution filter를 통과시키면서 feature를 추출.
  - 이 과정에서 이미지의 local pattern과 텍스처가 강조되며, 점차적으로 더 추상적인 특징으로 합쳐진다.

- ViT모델
  - 이미지를 여러개의 작은 패치로 나누고, 각 패치를 독립적인 입력데이터로 처리.
  - 이 패치들은 일련의 Transformer블록을 통과하면서, 각 패치가 이미지의 다른 부분과 어떻게 상호작용하는지를 학습.
  - 이는 Transformer가 자연어 처리에서 문장 내 단어 간의 관계를 학습하는 것과 유사한 방식

In [ ]:
import torch
import torchvision.transforms as transforms
from PIL import Image
import io


# 이미지로드
from google.colab import files

uploaded = files.upload()

# 첫 번째 업로드된 파일의 이름 가져오기
file_name = next(iter(uploaded))

# 이미지 열기
image = Image.open(io.BytesIO(uploaded[file_name]))   # io.BytesIO를 사용하여 업로드된 파일 데이터를 바이너리 스트림으로 변환하고, 이를 Image.open 함수에 전달

# CNN모델을 위한 전처리
cnn_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229, 0.224, 0.225])
])
cnn_preprocessed_img = cnn_transforms(image)

# ViT모델을 위한 전처리
vit_transforms = transforms.Compose([
    transforms.Resize((224,224)),   # ViT에 맞는 크기로 조정
    transforms.ToTensor(),    # ViT는 일반적으로 입력 이미지를 정규화하지 않는다.

])
vit_preprocessed_img = vit_transforms(image)

# 이미지를 패치로 나누는 과정(ViT)
patch_size = 16 # 16x16 pixel patch크기
patches = vit_preprocessed_img.unfold(1,patch_size, patch_size).unfold(2, patch_size, patch_size)
patches = patches.contiguous().view(-1,3,patch_size, patch_size)

# CNN과 ViT의 전처리된 이미지 출력
print(f"CNN Preprocessed Image Shape : {cnn_preprocessed_img.shape}")
print(f"ViT Patches Shape : {patches.shape}")

Saving andy-holmes-LUpDjlJv4_c-unsplash.jpg to andy-holmes-LUpDjlJv4_c-unsplash (5).jpg
CNN Preprocessed Image Shape : torch.Size([3, 224, 224])
ViT Patches Shape : torch.Size([196, 3, 16, 16])


설명)
- cnn_transform :  CNN모델을 위한 전처리 파이프라인 정의
  - CNN은 이미지를 정규화하고 중앙에서 자른다.

- vit_transform : ViT모델을 위한 전처리 파이프라인 정의.
  - ViT는 이미지를 패치로 나누는 추가 단계를 포함.
  - 이 패치들은 Transform의 입력으로 사용되며, 각 패치는 이미지의 일부분을 나타낸다.



In [ ]:
import torch
import torchvision.transforms as transforms
from torch.nn.functional import interpolate



# 임의의 이미지 데이터 생성(3채널, 256x256 크기)
random_image = torch.rand((3,256,256))

# CNN모델을 위한 전처리
cnn_transforms = transforms.Compose([
    transforms.Resize(224), # 224x224 크기로 리사이즈
    transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229, 0.224, 0.225])
])
cnn_preprocessed_img = cnn_transforms(random_image)

# ViT모델을 위한 전처리
vit_transforms = transforms.Compose([
    transforms.Resize((224,224)),   # ViT에 맞는 크기로 조정
    #transforms.ToTensor(),    # ViT는 일반적으로 입력 이미지를 정규화하지 않는다.

])
vit_preprocessed_img = vit_transforms(random_image)

# 이미지를 패치로 나누는 과정(ViT)
patch_size = 16 # 16x16 pixel patch크기
num_patches = (224 // patch_size) ** 2 # 패치의 개수
patches = vit_preprocessed_img.unfold(1,patch_size, patch_size).unfold(2, patch_size, patch_size)
patches = patches.contiguous().view(-1,3,patch_size, patch_size)

# CNN과 ViT의 전처리된 이미지 출력
print(f"CNN Preprocessed Image Shape : {cnn_preprocessed_img.shape}")
print(f"ViT Patches Shape : {patches.shape}")

CNN Preprocessed Image Shape : torch.Size([3, 224, 224])
ViT Patches Shape : torch.Size([196, 3, 16, 16])
